<a href="https://colab.research.google.com/github/Falenbayu/-Proyek-Klasifikasi-Gambar-Batu-Gunting-Kertas-Falen/blob/main/Proyek_Akhir_Klasifikasi_Gambar_Rockpaperscissors_Falen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Name        : FALEN BAYU ADHETYA
## Username    : falen_bayu
## Email       : falenbayu@gmail.com